# Extract

Load input data

In [ ]:
import os
import glob
import pandas as pd

SOURCE_HOST = "tau"


def read_csvs(directory: str) -> pd.DataFrame:
    files = sorted(glob.glob(os.path.join(directory, "*.csv")))
    df_list = [pd.read_csv(file, index_col=0, parse_dates=True) for file in files]
    return pd.concat(df_list).sort_index()


df = read_csvs(f"../../input_data/{SOURCE_HOST}/tracepoints/net/")

In [ ]:
df = df["2025-01-24":]  # discard IBD data

In [ ]:
# Any filter* messages?
spv_msgs = [msg for msg in df.msg_type.unique().tolist() if msg.startswith("filter")]
print(spv_msgs)

# investigate SPV peer
display(df[df["msg_type"] == "filterload"])
spv_peers = df[df["msg_type"] == "filterload"]["peer_addr"].unique().tolist()
for spv_peer in spv_peers:
    display(df[df["peer_addr"] == spv_peer])

# Inspect, Develop

In [ ]:
# df_backup = df.copy()

In [ ]:
df.info()

# Transform

In [ ]:
import pandas as pd

# from pandarallel import pandarallel

# pandarallel.initialize(progress_bar=True)


# Group by 'peer_id' and 'peer_addr', then aggregate the required metrics
result = (
    df.groupby(["peer_id", "peer_addr"])
    .apply(
        lambda g: pd.Series(
            {
                "connection_time": g.index.max() - g.index.min(),
                "in_size": g.loc[g["flow"] == "in", "size"].sum(),
                "out_size": g.loc[g["flow"] == "out", "size"].sum(),
                "total_size": g["size"].sum(),
                "in_count": g.loc[g["flow"] == "in", "size"].sum(),
                "out_count": g.loc[g["flow"] == "out", "size"].sum(),
                "total_count": g["size"].sum(),
            }
        )
    )
    .reset_index()
)

# # Suppose your dataframe is called df. First reset the index:
# df = df.reset_index()  # so that 'timestamp' is a column, not the index

# # Group and aggregate
# grouped = df.groupby(["peer_id", "peer_addr"], dropna=False)

# result = grouped.agg(
#     first_timestamp=("timestamp", "min"),
#     last_timestamp=("timestamp", "max"),
#     in_size=("size", lambda s: s[df.loc[s.index, "flow"] == "in"].sum()),
#     out_size=("size", lambda s: s[df.loc[s.index, "flow"] == "out"].sum()),
#     total_size=("size", "sum"),
#     in_count=("size", lambda s: s[df.loc[s.index, "flow"] == "in"], "count"),
#     out_count=("size", lambda s: s[df.loc[s.index, "flow"] == "out"], "count"),
#     total_count=("size", "count"),
# ).reset_index()

# # Compute the connection_time from first and last timestamps
# result["connection_time"] = result["last_timestamp"] - result["first_timestamp"]

# # print(result)

In [ ]:
import pandas as pd

# result.info()

# 1. Filter rows where connection_time < 1 hour
one_hour = pd.Timedelta(hours=1)
filtered = result[result["connection_time"] >= one_hour].copy()

duration_seconds = filtered["connection_time"].dt.total_seconds()
duration_hours = filtered["connection_time"].dt.total_seconds() / 3600

filtered["bw_in"] = filtered["in_size"] / duration_seconds
filtered["bw_out"] = filtered["out_size"] / duration_seconds
filtered["bw_total"] = filtered["total_size"] / duration_seconds

filtered["mph_in"] = filtered["in_count"] / duration_hours
filtered["mph_out"] = filtered["out_count"] / duration_hours
filtered["mph_ratio_in_out"] = filtered["mph_in"] / filtered["mph_out"]
filtered["mph_total"] = filtered["total_count"] / duration_hours

# 3. Sort by connection_time in ascending order
filtered = filtered.sort_values(by="mph_ratio_in_out", ascending=False)

print(len(filtered))
filtered

In [ ]:
import matplotlib.pyplot as plt

metric = "bw_total"

# 1. Sort by bw_total
filtered_sorted = filtered.sort_values(metric).reset_index(drop=True)

# 2. Plot the sorted values
plt.figure()
plt.plot(filtered_sorted[metric], marker="o", linestyle="")  # scatter-like
# or use plt.scatter(range(len(filtered_sorted)), filtered_sorted['bw_total'])

plt.xlabel("Row index after sorting")
plt.title(metric)
plt.yscale("log")
plt.show()

# Share analysis

In [ ]:
import pandas as pd
import numpy as np

# If your data is not exactly aligned with these columns,
# you'll need to rename or adapt accordingly:
# Columns assumed: peer_id, peer_addr, flow, msg_type, size, and a DateTimeIndex 'timestamp'.


# -------------------------------------------------------
# 2) First, group by (peer_id, peer_addr)
#    and compute the connection duration
# -------------------------------------------------------
group_cols = ["peer_id", "peer_addr"]


# We'll define a helper to calculate the duration
def connection_duration(g):
    return g.index.max() - g.index.min()


# Compute durations for each group
durations = df.groupby(group_cols).apply(connection_duration)

# durations is a Series with (peer_id, peer_addr) as index, and Timedelta as the value.
# Keep only groups > 1 hour
valid_pairs = durations[durations > pd.Timedelta(hours=1)].index

# Filter the main DataFrame to only include those valid pairs
df_filtered = df[df.set_index(group_cols).index.isin(valid_pairs)]


# -------------------------------------------------------
# 3) Discard all rows where flow == 'out'
#    i.e., keep only inbound messages
# -------------------------------------------------------
df_inbound = df_filtered[df_filtered["flow"] != "out"]


# -------------------------------------------------------
# 4) Now, for the remaining inbound-only rows, we want
#    to group by (peer_id, peer_addr) and msg_type
#    to compute:
#      - Count of each msg_type
#      - The share of each msg_type
# -------------------------------------------------------
# First, compute total counts per group & msg_type
counts = df_inbound.groupby(group_cols + ["msg_type"]).size().reset_index(name="count")

# Next, compute the total count *per group* so we can get the share
group_sums = counts.groupby(group_cols)["count"].transform("sum")

# Add a 'share' column to counts
counts["share"] = counts["count"] / group_sums

# Now we have a table with columns:
#   peer_id, peer_addr, msg_type, count, share
# If we want each msg_type's share as its own column, we can pivot.
result = counts.pivot_table(
    index=group_cols, columns="msg_type", values="share", fill_value=0
)

# Optionally rename columns to have a suffix "_share"
result = result.add_suffix("_share").reset_index()

# 'result' is now a DataFrame:
#   peer_id, peer_addr, <msg_type1>_share, <msg_type2>_share, ...
# For only the pairs that had >1 hour connections and inbound messages.


# -------------------------------------------------------
# 5) (Optional) If you also want the absolute counts in the same table,
#    you can pivot on the 'count' as well, or merge them
# -------------------------------------------------------
counts_pivot = counts.pivot_table(
    index=group_cols, columns="msg_type", values="count", fill_value=0
)
counts_pivot = counts_pivot.add_suffix("_count").reset_index()

# Merge shares and counts if desired
final = pd.merge(result, counts_pivot, on=group_cols, how="left")

# 'final' now has both share and count columns for each msg_type.
# e.g.  peer_id, peer_addr, getheaders_share, headers_share, ping_share, ...
#       getheaders_count, headers_count, ping_count, ...

print(final)

In [ ]:
final

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Sort your DataFrame by inv_share, tx_share (example)
# df_sorted = final.sort_values(by=["inv_share", "ping_share"])
df_sorted = final.sort_values(by=["getdata_share", "ping_share"])

# 2) Select columns that end with "_share"
share_cols = [col for col in df_sorted.columns if col.endswith("_share")]
df_shares = df_sorted[share_cols].reset_index(drop=True)

# 3) Filter out columns whose minimum value ≤ 0.01
valid_cols = [col for col in df_shares.columns if df_shares[col].max() > 0.001]
df_shares_filtered = df_shares[valid_cols]

# 4) Generate a color list using "tab20"
import matplotlib.cm as cm

cmap = cm.get_cmap("tab20")  # tab20 has 20 distinct colors
n_colors = len(valid_cols)

# We'll cycle through the 0..1 range in small increments if we have many columns
colors = [cmap((i % 20) / 19.0) for i in range(n_colors)]

# 5) Plot a stacked area chart with the filtered columns
ax = df_shares_filtered.plot.area(stacked=True, figsize=(12, 6), color=colors)
ax.legend(bbox_to_anchor=(0.5, 1.1), loc="upper center", ncol=5, title="", fontsize=6)

ax.set_xlabel("Row index (each row is a peer connection)")
ax.set_ylabel("Share")
ax.set_title("Stacked Area Chart of Message Type Shares (Filtered)")

plt.tight_layout()
plt.show()

# IP addresses of sorted

In [ ]:
df_sorted.peer_addr.head(50)

In [ ]:
df_sorted.columns

# Investigate zero-inv nodes

In [ ]:
final2 = final[final["inv_count"] == 0]
# 1) Sort your DataFrame by inv_share, tx_share (example)
df_sorted = final2.sort_values(by=["getheaders_share", "pong_share"])

# 2) Select columns that end with "_share"
share_cols = [col for col in df_sorted.columns if col.endswith("_share")]
df_shares = df_sorted[share_cols].reset_index(drop=True)

# 3) Filter out columns whose minimum value ≤ 0.01
valid_cols = [col for col in df_shares.columns if df_shares[col].max() > 0.001]
# valid_cols = [col for col in df_shares.columns ]
print(valid_cols)
df_shares_filtered = df_shares[valid_cols]


# 4) Generate a color list using "tab20"
import matplotlib.cm as cm

cmap = cm.get_cmap("tab20")  # tab20 has 20 distinct colors
n_colors = len(valid_cols)

# We'll cycle through the 0..1 range in small increments if we have many columns
colors = [cmap((i % 20) / 19.0) for i in range(n_colors)]

# 5) Plot a stacked area chart with the filtered columns
ax = df_shares_filtered.plot.area(stacked=True, figsize=(12, 6), color=colors)
ax.legend(bbox_to_anchor=(0.5, 1.1), loc="upper center", ncol=5, title="", fontsize=6)

ax.set_xlabel("Row index (each row is a peer connection)")
ax.set_ylabel("Share")
ax.set_title("Stacked Area Chart of Message Type Shares (Filtered)")

plt.tight_layout()
plt.show()

# Verack count

In [ ]:
final2["getdata_count"].value_counts()

## Step 1: Discard useless data

Strip unnecessary data:

- Tracepoint data includes `peer_id`, `peer_conn_type`, `peer_addr`, `flow`
(traffic directory, i.e. in- or outbound), `msg_type` and `size`. Of these, only
`flow`, `msg_type` and `size` are retained in their original form. A new `ipv6`
column is introduced to indicate whether a message was sent via IPv4 or IPv6,
since the version affects IP header sizes used for the traffic estimate.

- Systemd IP accounting data includes rows for `IPIngressPackets`,
`IPEgressPackets`, `IPIngressBytes`, and `IPEgressBytes`. Packet data is
discarded and byte data is converted from absolute to relative (i.e. from bytes
since measurement was started to bytes since previous row/sample).

In [ ]:
from pandarallel import pandarallel
import numpy as np

pandarallel.initialize(progress_bar=True)


df_tp = df_tp.dropna()
df_tp["ipv6"] = df_tp["peer_addr"].parallel_apply(lambda x: True if "[" in x else False)
df_tp = df_tp[["ipv6", "flow", "msg_type", "size"]]

df_emp = df_emp.dropna()
df_emp = df_emp[["IPIngressBytes", "IPEgressBytes"]]
df_emp = df_emp.diff()[1:]

## Step 2: Estimate TCP/IP traffic from message sizes

TCP/IP traffic is estimated using the following assumptions:
- MTU size is 1500 bytes (common default)
- Bitcoin protocol overhead is 24 bytes (4-byte magic, 12-byte command, 4-byte
  each for payload length and checksum)
- TCP header size of 32 bytes, comprising 20-byte minimum TCP header size plus 10-byte timestamps option (used by default by the Linux kernel to make real-time round-trip measurements) and two padding bytes to align options to 32-bit boundaries
- IPv4 and v6 header sizes of 20 and 40 bytes (default)

The estimate uses the following approach. First, the application-level message
size is computed by adding the Bitcoin P2P message overhead to the message size.
Next, the number of TCP segments is computed by dividing the application-level
size obtained during the previous step by the maximum segment size (which
corresponds to the MTU minus TCP and IP headers) to compute the number of TCP
segments. Then, the total TCP/IP overhead is computed (number of segments times
TCP and IP header overhead). Moreover, the overhead of ACKs is estimated to be
half of the number of segments times the sum of IP and TCP header sizes, since
generally ACKs are sent for every two packets.  Finally, TCP/IP traffic is
estimated by combining the application-level message size with the total TCP/IP
and ACK overhead.

Next, empirical TCP/IP measurements obtained via systemd accounting are combined
with the estimate so the latter can be validated.

In [ ]:
import math
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)


def estimate_network_traffic(row):
    MAX_MTU_SIZE = 1500
    BITCOIN_PROTOCOL_OVERHEAD = 24
    TCP_HEADER_SIZE = 32
    IP_HEADER_SIZE = 40 if row["ipv6"] else 20
    ACK_RATIO = 2
    MSS = MAX_MTU_SIZE - IP_HEADER_SIZE - TCP_HEADER_SIZE
    bitcoin_message_size = row["size"] + BITCOIN_PROTOCOL_OVERHEAD
    num_segments = math.ceil(bitcoin_message_size / MSS)
    tcpip_overhead = num_segments * (IP_HEADER_SIZE + TCP_HEADER_SIZE)
    ack_overhead = (num_segments / ACK_RATIO) * (IP_HEADER_SIZE + TCP_HEADER_SIZE)
    return bitcoin_message_size + tcpip_overhead + ack_overhead


df_tp["net_size"] = df_tp.parallel_apply(estimate_network_traffic, axis=1)

### Step 3: Aggregate data (to hourly and daily granularity)

First, the dataframe contaiing empirical data from systemd's IP accounting is
pivoted so it can be aggregated.

Next, the pivoted df and the tracepoint df are aggregated to produce hourly and
daily data.

In [ ]:
df_emp = (
    df_emp.rename(columns={"IPIngressBytes": "in", "IPEgressBytes": "out"})[
        ["in", "out"]
    ]
    .stack()
    .rename("net_size")
    .reset_index()
    .rename(columns={"level_1": "flow"})
    .set_index("timestamp")
)


def agg_sum(df, cols, freq, data="net_size"):
    """Aggregate 'data' col based on datetime index with frequency 'freq', using
    summation using 'cols' as differentiator."""
    df_tmp = df.copy()
    df_tmp.index = df_tmp.index.floor(freq)
    df_result = (
        df_tmp.groupby(["timestamp"] + cols)[data]
        .sum()
        .reset_index()
        .set_index("timestamp")
    )
    return df_result


dfs = {
    "est_hourly": agg_sum(df_tp, ["flow", "msg_type"], freq="1h"),
    "est_daily": agg_sum(df_tp, ["flow", "msg_type"], freq="1d"),
    "emp_hourly": agg_sum(df_emp, ["flow"], freq="1h"),
    "emp_daily": agg_sum(df_emp, ["flow"], freq="1d"),
}

## Step 4: Format aggregated data

Pivot `flow` column of dataframes to get `in` and `out` columns.

In [ ]:
def pivot(df, index, columns="flow", values="net_size"):
    """Pivot dataframe: keep 'index' as rows, 'columns' as columns and 'values'
    as values.  Set 'timestamp' as new index, fill missing values with zero and
    convert new cols to int."""

    return (
        df.reset_index()
        .pivot(
            index=index,
            columns=columns,
            values=values,
        )
        .rename_axis(None, axis=1)
        .reset_index()
        .set_index("timestamp")
        .fillna(0)
        .astype({"in": "int", "out": "int"})
    )


dfs = {
    "est_hourly": pivot(dfs["est_hourly"], ["timestamp", "msg_type"]),
    "est_daily": pivot(dfs["est_daily"], ["timestamp", "msg_type"]),
    "emp_hourly": pivot(dfs["emp_hourly"], ["timestamp"]),
    "emp_daily": pivot(dfs["emp_daily"], ["timestamp"]),
}

## Step 5: Sanitize data

Whenever the `nix-bitcoin-monitor` systemd service (which performs the data
collection) is restarted, the IP accounting counters are reset to zero. As a
result, `diff()`ing consecutive readings is going to break (think large valule
in previous row followed by small value in next row, leading to negative
values). This is addressed by setting values smaller than zero to zero.

In [ ]:
for name, df in dfs.items():
    if not name.startswith("emp_"):
        continue
    for row in ["in", "out"]:
        df.loc[df[row] < 0, row] = 0
    dfs[name] = df

# Load

Store transformation results

In [ ]:
from pathlib import Path

data_dir = Path("data")
if not data_dir.exists():
    data_dir.mkdir()

for name, df in dfs.items():
    df.to_csv(f"data/data_{name}.csv.bz2", compression="bz2")